# Property Getters and Setters

https://docs.python.org/3/library/functions.html#property  
https://docs.python.org/3/howto/descriptor.html  
https://stackoverflow.com/questions/6618002/using-property-versus-getters-and-setters  
https://stackoverflow.com/questions/38118264/how-can-i-retrieve-the-docstring-for-a-property-of-a-python-class-instance  
https://www.blog.pythonlibrary.org/2016/06/10/python-201-what-are-descriptors/


In [137]:
import pprint
import inspect

pp = pprint.PrettyPrinter(indent=4)


## What's the difference between a Python “property” and “attribute”?

https://stackoverflow.com/questions/7374748/whats-the-difference-between-a-python-property-and-attribute  
https://www.python-course.eu/python3_properties.php  

Properties are a special kind of attribute. Basically, when Python encounters the following code:

    spam = SomeObject()
    print(spam.eggs)

it looks up `eggs` in spam, and then examines `eggs` to see if it has a` __get__`, `__set__`, or `__delete__` method - if it does, it's a property. If it is a property, instead of just returning the `eggs` object (as it would for any other attribute) it will call the `__get__` method (since we were doing lookup) and return whatever that method returns.

With a property you have complete control on its getter, setter and deleter methods, which you don't have (if not using caveats) with an attribute.

1. Property is a more convenient way to do data encapsulation.

    Example: If your have a public attribute `length` of Object, later on, your project requires you to encapsulate it, i.e: change it to private and provide getter and setter => you have to change many of the codes you wrote before:

        #Old codes
        obj1.length = obj1.length+obj2.length
        
        #New codes(Using private attibutes and getter and setter)
        obj1.set_length(obj1.get_length()+obj2.get_length())  # => this is ugly
        

    If you use `@property` and `@length.setter` => you don't need to change those old codes

2. A property can access and encapsulate multiple attributes. In the Person example below  `__physic_health` and` __mental_health` are private and can not be accessed directly from out side, the only way outside class interact with them is through a property condition

In [138]:
class Person:
  def __init__(self, name, physic_health, mental_health):
    self.name=name
    self.__physic_health = physic_health #physic_health is real value in range [0, 5.0]
    self.__mental_health = mental_health #mental_health is real value in range [0, 5.0]

  @property
  def condition(self):
    health = self.__physic_health+self.__mental_health
    if(health < 5.0):
      return "I feel bad!"
    elif health < 8.0:
      return "I am ok!"
    else:
      return "Great!"

p = Person('me',2,2)
print(p.name)
# print(p.__physic_health) # won't work
print(p.condition)

me
I feel bad!


We often have a function connected to class attribute. For instance i need to read file once and keep content assigned to the attribute so the value is cached. Using a property will cause attribute's value refresh each time you access it:

In [139]:
class Misc():

    @property
    def test(self):
        print('func running')
        return 'func value'

cl = Misc()
print(cl.test)
print(cl.test)

func running
func value
func running
func value


## Descriptor Design Pattern for Properties

http://buildingskills.itmaybeahack.com/book/python-2.6/html/p03/p03c05_properties.html  
https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work  
https://docs.python.org/3/howto/descriptor.html  

A Descriptor is a class which provides the detailed get, set and delete control over an attribute of another object. This allows you to define attributes which are fairly complex objects in their own right. The idea is that we can use simple attribute references in a program, but those simple references are actually method functions of a descriptor object.

The Descriptor design pattern has two parts: an Owner and an attribute Descriptor. The Owner is usually a relatively complex object that uses one or more Descriptors for its attributes. A Descriptor class defines get, set and delete methods for a class of attributes of the Owner. Each Descriptor object manages a specific attribute. Note that Descriptors will tend to be reusable, generic classes of attributes. The Owner can have multiple instances of each Descriptor class to manage attributes with similar behaviors. Each Descriptor object is a unique instance of a Descriptor class, bound to a distinct attribute name when the Owner class is defined.'

The `property()` function returns a special descriptor object.
These act as decorators too. They return a new property object.
Remember, that the `@decorator` syntax is just syntactic sugar; the syntax:

    @property
    def foo(self): 
        return self._foo

really means the same thing as

    def foo(self): 
        return self._foo
    foo = property(foo)

so `foo` the function is replaced by `property(foo)`, which we saw above is a special object. Then when you use `@foo.setter()`, what you are doing is call that `property().setter` method, which returns a new copy of the property, but this time with the setter function replaced with the decorated method.


In [140]:
print(property())
print(property().getter)
print(property().setter)
print(property().deleter)


<built-in method getter of property object at 0x00000000062EBA98>
<built-in method setter of property object at 0x00000000062EBA98>
<built-in method deleter of property object at 0x00000000062EBA98>


https://stackoverflow.com/questions/6193556/how-do-python-properties-work

In order for `@properties` to work properly the class needs to be a subclass of `object`. when the class is not a subclass of `object` then the first time you try access the setter it actually makes a new attribute with the shorter name instead of accessing through the setter. 

## Sachin's blog

https://codesachin.wordpress.com/2016/06/09/the-magic-behind-attribute-access-in-python/  
https://stackoverflow.com/questions/25440694/whats-the-purpose-of-dictproxy
    
Most people know just one thing when it comes to attribute access – the dot ‘.’ (as in `x.some_attribute`). In simple terms, attribute access is the way you retrieve an object linked to the one you already have. To someone who uses Python without delving too much into the details, it may seem pretty straightforward. However, under the hood, theres a lot that goes on for this seemingly trivial task.    

Every object in Python has an attribute denoted by `__dict__`. This dictionary/dictionary-like object contains all the attributes defined for the object itself. It maps the attribute name to its value.

In [141]:
 class C(object):
    x = 4

c = C()
c.y = 5
c.__dict__

{'y': 5}

Notice how 'x' is not in `c.__dict__`. The reason for this is simple enough. While y was defined for the object c, x was defined for its class (C). Therefore, it will appear in the `__dict__` of C. In fact, C‘s `__dict__` contains a lot of other keys too (including '`__dict__`'):


In [142]:
c.__class__.__dict__['x']

4

In [143]:
C.__dict__

mappingproxy({'__module__': '__main__',
              'x': 4,
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

In [144]:
print(type(c.__class__.__dict__))
c.__class__.__dict__

<class 'mappingproxy'>


mappingproxy({'__module__': '__main__',
              'x': 4,
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

We will look at `mappingproxy` below. It used to be called `dictproxy` in older documentation.

The `__dict__` of an object is simple enough to understand. It behaves like a Python dict, and is one too.
The `__dict__` of a class however, is not that straight-forward. Its actually an object of a class called `mappingproxy`. `mappingproxy` is a special class whose objects behave like normal dicts, but they differ in some key behaviours. It does not support assignment and is not writeable.

You cannot set a key in a dictproxy directly (`C.__dict__['x'] = 4` does not work). You can accomplish the same using `C.x = 6` however, since the internal behaviour then is different. You cannot set the `__dict__` attribute itself either(`C.__dict__ = {}` does not work).

A descriptor is an object that has atleast one of the following magic methods in its attributes: `__get__`, `__set__` or `__delete__`.
Descriptors can help you define the behaviour of an object’s attribute in Python. With each of the magic methods just mentioned, you implement how the attribute (‘described’ by the descriptor) will be retrieved, set and deleted in the object respectively.    

There are two types of descriptors:  Non-Data Descriptors (only have `__get__` defined) and Data Descriptors (more than just `__get__` defined).

Descriptors are used for a lot of attribute and method related functionality in Python, including static methods, class methods and properties. Using descriptors, you can gain better control over how attributes and methods of a class/its objects are accessed - including defining some ‘behind the scenes’ functionality like logging.

Now lets look at the high-level rules governing attribute access (getting) in Python.
The workflow is as follows:

1. If attrname is a special (i.e. Python-provided) attribute for objectname, return it.
1. Check `objectname.__class__.__dict__` for attrname. If it exists and is a data-descriptor, return the descriptor result. Search all bases of `objectname.__class__` for the same case.
1. Check `objectname.__dict__` for attrname, and return if found. If objectname is a class, search its bases too. If it is a class and a descriptor exists in it or its bases, return the descriptor result.
1. Check `objectname.__class__.__dict__` for attrname. If it exists and is a non-data descriptor, return the descriptor result. If it exists, and is not a descriptor, just return it. If it exists and is a data descriptor, we shouldn’t be here because we would have returned at point 2. Search all bases of `objectname.__class__` for same case.
1. Raise AttributeError

Rules for setting attributes

1. Check `objectname.__class__.__dict__` for attrname. If it exists and is a data-descriptor, use the descriptor to set the value. Search all bases of `objectname.__class__` for the same case.
1. Insert something into `objectname.__dict__` for key "attrname".

`__slots__` is a way to disallow objects from having their own `__dict__` in Python. This means, that if you define `__slots__` in a Class, then you cannot set arbitrary attributes(apart from the ones mentioned in the ‘slots’) on its objects.



## Python docs

https://docs.python.org/3/library/functions.html#property  
https://docs.python.org/3/reference/datamodel.html#implementing-descriptors

`class property(fget=None, fset=None, fdel=None, doc=None)`

Return a property attribute.

`fget` is a function for getting an attribute value.   
`fset` is a function for setting an attribute value.   
`fdel` is a function for deleting an attribute value.  
`doc` creates a docstring for the attribute.

A typical use is to define a managed attribute x as shown in the code below.

If c is an instance of C, `c.x` will invoke the getter, `c.x = value` will invoke the setter and `del(c.x)` the deleter.

If given, `doc` will be the docstring of the property attribute. Otherwise, the property will copy fget’s docstring (if it exists).

This makes it possible to create read-only properties easily using `property()` as a decorator.
In the code below `@property` decorator turns the `voltage()` method into a “getter” for a read-only attribute with the same name, and it sets the docstring for voltage to “Get the current voltage.”


Note that because of the lookup rules (see below in Sachin's blog), to get an attribute docstring [you must use](https://stackoverflow.com/questions/38118264/how-can-i-retrieve-the-docstring-for-a-property-of-a-python-class-instance) `__class_`_ to get the class of the instance and then the property, and finally the `__doc__`: `p.__class__.voltage.__doc__` or use `type` to fetch the class.


In [145]:
class C:
    def __init__(self):
        self._x = None

    def getx(self):
        return self._x

    def setx(self, value):
        self._x = value

    def delx(self):
        del self._x

    x = property(getx, setx, delx, "I'm the 'x' property.")

attributes = inspect.getmembers(C, lambda a:not(inspect.isroutine(a)))
# pp.pprint(attributes)
# get rid of dunders
pp.pprint([a for a in attributes if not(a[0].startswith('__') and a[0].endswith('__'))])

[('x', <property object at 0x000000000652AA98>)]


In [146]:
print('Create class and print default value of x, using the function')
b = C()
print(b.getx())
print('Assign the string "a" to x and print using the function')
b.setx('a')
print(b.getx())
print('Assign the string "A" to x and print using the property attribute')
b.x = 'A'
print(b.x)
print('Print the attribute docstring,')
print(b.__class__.x.__doc__)
print()
print('Delete x and print using the function / property attribute')
# b.delx()
del(b.x)
# print(b.x) # !! will give an error

Create class and print default value of x, using the function
None
Assign the string "a" to x and print using the function
a
Assign the string "A" to x and print using the property attribute
A
Print the attribute docstring,
I'm the 'x' property.

Delete x and print using the function / property attribute


In [147]:
class Parrot:
    def __init__(self):
        self._volts = 100000

    @property
    def voltage(self):
        """Get the current voltage."""
        return self._volts

p = Parrot()
print(p.voltage)
# this will not work because the setter is not implemented
# p.voltage = 0

print(p.__class__.voltage.__doc__)
print(type(p).voltage.__doc__)

100000
Get the current voltage.
Get the current voltage.


A property object has `getter`, `setter`, and `deleter` methods usable as decorators that create a copy of the property with the corresponding accessor function set to the decorated function. This is best explained with the code below. 
This code is exactly equivalent to the first example above. _Be sure to give the additional functions the same name as the original property_ (x in this case.)

Why use decorators? See the long discussion here: https://stackoverflow.com/questions/6618002/using-property-versus-getters-and-setters 

```
The reason is that all attributes are public in Python. Starting names with an underscore or two 
is just a warning that the given attribute is an implementation detail that may not stay the same 
in future versions of the code. It doesn't prevent you from actually getting or setting that attribute. 
Therefore, standard attribute access is the normal, Pythonic way of, well, accessing attributes.

The advantage of properties is that they are syntactically identical to attribute access, so you can 
change from one to another without any changes to client code. You could even have one version of a 
class that uses properties (say, for code-by-contract or debugging) and one that doesn't for 
production, without changing the code that uses it. At the same time, you don't have to write
getters and setters for everything just in case you might need to better control access later.
```

Changed in version 3.5: The docstrings of property objects are now writeable.

In [148]:
class C:
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

In [149]:
print('Create class and print default value of x, using the function')
b = C()
print(b.x)
print('Assign the string "a" to x and print using the function')
b.x = 'a'
print(b.x)
print('Assign the string "A" to x and print using the property attribute')
b.x = 'A'
print(b.x)
print('Print the attribute docstring,')
print(b.__class__.x.__doc__)
print()
print('Delete x and print using the function / property attribute')
# b.delx()
del(b.x)
# print(b.x) # !! will give an error


Create class and print default value of x, using the function
None
Assign the string "a" to x and print using the function
a
Assign the string "A" to x and print using the property attribute
A
Print the attribute docstring,
I'm the 'x' property.

Delete x and print using the function / property attribute


## Python and [module versions, and dates](https://github.com/rasbt/watermark)

In [150]:
# to get software versions
# https://github.com/rasbt/watermark
# https://github.com/rasbt/watermark/blob/master/docs/watermark.ipynb
# you only need to do this once
# pip install watermark
# conda install -c conda-forge watermark

%load_ext watermark
%watermark -v -m -p numpy,scipy -g 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.1
IPython 7.2.0

numpy 1.15.4
scipy 1.1.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 7
machine    : AMD64
processor  : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
CPU cores  : 8
interpreter: 64bit
Git hash   : eba386dcbe704dc355e1a01920c000703d510811
